In [1]:
import time
import tempfile
import pandas as pd
import subprocess as sp

In [2]:
commands = [
    'sleep 3',    # 0
    'ls -l /',    # 1
    'find /',     # 2
    'sleep 4',    # 3
    'find /usr',  # 4
    'date',       # 5
    'sleep 5',    # 6
    'uptime'      # 7
]

Creating an empty times dictionary for commands and their corresponding times, later to be filled in by the while loop, and opening a process for each command.

In [3]:
times = {}
processes = []

start_time = time.time()

Standard output goes into a tempfile. Have to use a tempfile instead of setting **stdout** and **stderr** to PIPE because the 'find /' command causes subprocess hanging. For that one command, PIPE didn't work.

In [4]:
for command in commands:
    processes.append(sp.Popen(command, shell=True, 
                              stdout=tempfile.TemporaryFile()))

8 processes open, one for each command:

In [5]:
processes

While loop runs the processes. After each process is finished, the stop time is recorded in the times dictionary, and the process is popped from the list.

In [7]:
while len(processes) > 0:
    for i, process in enumerate(processes):
          if process.poll() is not None:
              times[process.args] = (time.time() - start_time)
              processes.pop(i)

In [9]:
times

{'date': 0.0854341983795166,
 'find /': 30.558664083480835,
 'find /usr': 1.2270500659942627,
 'ls -l /': 0.08518719673156738,
 'sleep 3': 3.0165462493896484,
 'sleep 4': 4.031415224075317,
 'sleep 5': 5.043739080429077,
 'uptime': 0.08545517921447754}

--------

In [10]:
total_elapsed = print(sum(times.values()))

44.1334912776947


In [11]:
times_series = pd.Series(times)
times_series.describe()

count     8.000000
mean      5.516686
std      10.300347
min       0.085187
25%       0.085450
50%       2.121798
75%       4.284496
max      30.558664
dtype: float64

**Total elapsed time:** 44.13s 

**Average:** 5.51s 

**Minimum:** 0.08 - 'ls -l /'

**Maximum:** 30.55s - 'find /'